In [1]:
!git clone https://github.com/dmarx/Multi-Modal-Comparators
%cd 'Multi-Modal-Comparators'
!pip install poetry
!poetry build
!pip install dist/mmc*.whl

# optional final step:
#poe napm_installs
!python src/mmc/napm_installs/__init__.py

import mmc
from mmc import loaders

from mmc.registry import REGISTRY
all_loaders = REGISTRY.find()
len(all_loaders)

Cloning into 'Multi-Modal-Comparators'...
remote: Enumerating objects: 490, done.
remote: Counting objects: 100% (132/132), done.
remote: Compressing objects: 100% (93/93), done.
remote: Total 490 (delta 64), reused 58 (delta 29), pack-reused 358
Receiving objects: 100% (490/490), 718.48 KiB | 2.21 MiB/s, done.
Resolving deltas: 100% (279/279), done.
/content/Multi-Modal-Comparators
     |████████████████████████████████| 175 kB 14.1 MB/s 
     |████████████████████████████████| 8.8 MB 65.3 MB/s 
     |████████████████████████████████| 91 kB 13.7 MB/s 
     |████████████████████████████████| 40 kB 7.9 MB/s 
     |████████████████████████████████| 54 kB 3.0 MB/s 
     |████████████████████████████████| 425 kB 56.8 MB/s 
     |████████████████████████████████| 48 kB 6.5 MB/s 
     |████████████████████████████████| 58 kB 6.7 MB/s 
     |████████████████████████████████| 58 kB 4.0 MB/s 
     |████████████████████████████████| 4.0 MB 47.8 MB/s 
     |████████████████████████████████| 461 k

2022-05-04 20:08:32.836 | DEBUG    | napm.config:config_path:28 - /root/.cache/napm/config.yaml
2022-05-04 20:08:32.836 | DEBUG    | napm.config:config_path:31 - Creating config file at /root/.cache/napm/config.yaml
2022-05-04 20:08:32.839 | DEBUG    | __main__:napm_pi_katcloob:18 - using napm to "install" katCLOOB
Cloning into '/root/.cache/napm/cloob'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 30 (delta 9), reused 27 (delta 6), pack-reused 0
Unpacking objects: 100% (30/30), done.
2022-05-04 20:08:33.218 | DEBUG    | napm.utils:gitclone:22 - 
2022-05-04 20:08:33.219 | DEBUG    | napm.config:config_path:28 - /root/.cache/napm/config.yaml
2022-05-04 20:08:33.221 | DEBUG    | napm.config:config_path:28 - /root/.cache/napm/config.yaml


2022-05-04 20:08:35.778 | DEBUG    | mmc.registry:register_model:38 - registering model: [clip - openai - RN50]
2022-05-04 20:08:35.779 | DEBUG    | mmc.registry:register_model:38 - registering model: [clip - openai - RN101]
2022-05-04 20:08:35.784 | DEBUG    | mmc.registry:register_model:38 - registering model: [clip - openai - RN50x4]
2022-05-04 20:08:35.785 | DEBUG    | mmc.registry:register_model:38 - registering model: [clip - openai - RN50x16]
2022-05-04 20:08:35.789 | DEBUG    | mmc.registry:register_model:38 - registering model: [clip - openai - RN50x64]
2022-05-04 20:08:35.791 | DEBUG    | mmc.registry:register_model:38 - registering model: [clip - openai - ViT-B/32]
2022-05-04 20:08:35.793 | DEBUG    | mmc.registry:register_model:38 - registering model: [clip - openai - ViT-B/16]
2022-05-04 20:08:35.797 | DEBUG    | mmc.registry:register_model:38 - registering model: [clip - openai - ViT-L/14]
2022-05-04 20:08:35.799 | DEBUG    | mmc.registry:register_model:38 - registering m

55

In [2]:
import torch
torch.cuda.mem_get_info() # [available, total] # global free and total GPU memory occupied

(16804216832, 17071734784)

In [3]:
import psutil
psutil.virtual_memory()

svmem(total=13617745920, available=12058550272, percent=11.4, used=1369784320, free=9010171904, active=1553973248, inactive=2593570816, buffers=151691264, cached=3086098432, shared=9629696, slab=202342400)

In [5]:
import torch
import gc

def check_gpu_mem():
    gc.collect()
    torch.cuda.empty_cache()
    return torch.cuda.mem_get_info()[0]

import time

st = time.time()

errors = []
memory_occupied = []
for ldr in all_loaders:
  try:
    model = ldr.load()
  except:
    print(f"error: {ldr}")
    errors.append(str(ldr))
    gc.collect()
    torch.cuda.empty_cache()
    time.sleep(2) # yeah, the GPU needs time to flush
    continue
  mem_loaded = check_gpu_mem()
  del model
  gc.collect()
  torch.cuda.empty_cache()
  time.sleep(2) # yeah, the GPU needs time to flush
  mem_released = check_gpu_mem()
  rec = str(ldr), mem_released - mem_loaded
  print((rec[0], rec[1] / 1024**2))
  memory_occupied.append(rec)
        
et = time.time()
print(et-st)

import pandas as pd
pd.DataFrame(memory_occupied)

('[clip - openai - RN50]', 358.0)


100%|███████████████████████████████████████| 278M/278M [00:03<00:00, 84.3MiB/s]


('[clip - openai - RN101]', 294.0)


100%|███████████████████████████████████████| 402M/402M [00:04<00:00, 97.4MiB/s]


('[clip - openai - RN50x4]', 424.0)


...

2022-05-04 21:12:35.103 | DEBUG    | mmc.loaders.fairsliploader:_napm_install:146 - using napm to "install" facebookresearch/SLIP
2022-05-04 21:12:35.106 | WARNING  | napm.pseudo_install:make_install_dir:21 - /root/.cache/napm/_envs/mmc/SLIP already exists
2022-05-04 21:12:35.235 | DEBUG    | napm.utils:gitclone:22 - 
2022-05-04 21:12:35.237 | DEBUG    | napm.pseudo_install:pseudoinstall_git_repo:62 - Added /root/.cache/napm/_envs/mmc/SLIP to sys.path
2022-05-04 21:12:35.240 | DEBUG    | napm.config:config_path:28 - /root/.cache/napm/_envs/mmc/config.yaml
2022-05-04 21:12:35.247 | DEBUG    | napm.config:config_path:28 - /root/.cache/napm/_envs/mmc/config.yaml
2022-05-04 21:12:35.253 | DEBUG    | napm.config:config_path:28 - /root/.cache/napm/_envs/mmc/config.yaml
2022-05-04 21:12:35.260 | DEBUG    | mmc.loaders.fairsliploader:model_factory_from_id:57 - SLIP_VITB16
2022-05-04 21:12:35.262 | DEBUG    | mmc.loaders.fairsliploader:load:163 - model_factory: <function SLIP_VITB16 at 0x7fa46e

  0%|          | 0.00/1.91G [00:00<?, ?B/s]

2022-05-04 21:14:16.339 | DEBUG    | mmc.loaders.fairsliploader:load:172 - {'ssl_emb_dim': 256, 'ssl_mlp_dim': 4096}


('[slip - facebookresearch - slip_base_cc12m_35ep]', 714.0)
3844.286712408066


,0,1
0,[clip - openai - RN50],375390208
1,[clip - openai - RN101],308281344
2,[clip - openai - RN50x4],444596224
3,[clip - openai - RN50x16],692060160
4,[clip - openai - RN50x64],1415577600
5,[clip - openai - ViT-B/32],385875968
6,[clip - openai - ViT-B/16],364904448
7,[clip - openai - ViT-L/14],952107008
8,[clip - openai - ViT-L/14@336px],952107008
9,[clip - mlfoundations - RN50--openai],421527552


In [22]:
import pandas as pd

In [6]:
errors

['[clip - facebookresearch - clip_small_25ep]',
 '[simclr - facebookresearch - simclr_small_25ep]',
 '[clip - facebookresearch - clip_base_25ep]',
 '[simclr - facebookresearch - simclr_base_25ep]',
 '[clip - facebookresearch - clip_large_25ep]',
 '[simclr - facebookresearch - simclr_large_25ep]',
 '[clip - facebookresearch - clip_base_cc3m_40ep]',
 '[clip - facebookresearch - clip_base_cc12m_35ep]']

In [9]:
df_mmc = pd.DataFrame(memory_occupied)

df_mmc.columns = ['model', 'vram_bytes']
df_mmc['vram_mb'] = df_mmc['vram_bytes'] / 1024**2

df_mmc['architecture'] = df_mmc['model'].apply(lambda x: x[1:-1].split(' - ')[0])
df_mmc['publisher'] = df_mmc['model'].apply(lambda x: x[1:-1].split(' - ')[1])
df_mmc['model_name'] = df_mmc['model'].apply(lambda x: x[1:-1].split(' - ')[2])

df_mmc = df_mmc[['publisher', 'architecture', 'model_name', 'vram_mb']]

In [18]:
df_mmc.to_markdown()

'|    | publisher        | architecture   | model_name                          |   vram_mb |\n|---:|:-----------------|:---------------|:------------------------------------|----------:|\n|  0 | openai           | clip           | RN50                                |       358 |\n|  1 | openai           | clip           | RN101                               |       294 |\n|  2 | openai           | clip           | RN50x4                              |       424 |\n|  3 | openai           | clip           | RN50x16                             |       660 |\n|  4 | openai           | clip           | RN50x64                             |      1350 |\n|  5 | openai           | clip           | ViT-B/32                            |       368 |\n|  6 | openai           | clip           | ViT-B/16                            |       348 |\n|  7 | openai           | clip           | ViT-L/14                            |       908 |\n|  8 | openai           | clip           | ViT-L/14@336px  

In [29]:
round2 = []
for m in errors:
  arch, pub, mid = m[1:-1].split(' - ')
  qry = {'architecture':arch, 'publisher':pub, 'id':mid}
  ldr = REGISTRY.find(**qry)[0]
  try:
    model = ldr.load()
  except Exception as e:
    print(ldr)
    print(e)
    continue
  mem_loaded = check_gpu_mem()
  del model
  gc.collect()
  torch.cuda.empty_cache()
  time.sleep(2) # yeah, the GPU needs time to flush
  mem_released = check_gpu_mem()
  print((rec[0], rec[1] / 1024**2))
  round2.append(rec)

[clip - mlfoundations - RN50--yfcc15m]
unexpected EOF, expected 832488 more bytes. The file might be corrupted.


2022-05-04 20:00:12.361 | DEBUG    | mmc.loaders.fairsliploader:_napm_install:146 - using napm to "install" facebookresearch/SLIP
2022-05-04 20:00:12.363 | WARNING  | napm.pseudo_install:make_install_dir:21 - /root/.cache/napm/_envs/mmc/SLIP already exists
2022-05-04 20:00:12.524 | DEBUG    | napm.utils:gitclone:22 - 
2022-05-04 20:00:12.526 | DEBUG    | napm.pseudo_install:pseudoinstall_git_repo:62 - Added /root/.cache/napm/_envs/mmc/SLIP to sys.path
2022-05-04 20:00:12.530 | DEBUG    | napm.config:config_path:28 - /root/.cache/napm/_envs/mmc/config.yaml
2022-05-04 20:00:12.538 | DEBUG    | napm.config:config_path:28 - /root/.cache/napm/_envs/mmc/config.yaml
2022-05-04 20:00:12.545 | DEBUG    | napm.config:config_path:28 - /root/.cache/napm/_envs/mmc/config.yaml
2022-05-04 20:00:12.549 | DEBUG    | mmc.loaders.fairsliploader:model_factory_from_id:57 - CLIP_VITS16
2022-05-04 20:00:12.551 | DEBUG    | mmc.loaders.fairsliploader:load:163 - model_factory: None


[clip - mlfoundations - RN50-quickgelu--yfcc15m]
unexpected EOF, expected 832488 more bytes. The file might be corrupted.


2022-05-04 20:00:17.338 | DEBUG    | mmc.loaders.fairsliploader:load:172 - {'ssl_emb_dim': 256, 'ssl_mlp_dim': 4096}
2022-05-04 20:00:17.341 | DEBUG    | mmc.loaders.fairsliploader:_napm_install:146 - using napm to "install" facebookresearch/SLIP
2022-05-04 20:00:17.348 | WARNING  | napm.pseudo_install:make_install_dir:21 - /root/.cache/napm/_envs/mmc/SLIP already exists
2022-05-04 20:00:17.491 | DEBUG    | napm.utils:gitclone:22 - 
2022-05-04 20:00:17.492 | DEBUG    | napm.pseudo_install:pseudoinstall_git_repo:62 - Added /root/.cache/napm/_envs/mmc/SLIP to sys.path
2022-05-04 20:00:17.498 | DEBUG    | napm.config:config_path:28 - /root/.cache/napm/_envs/mmc/config.yaml
2022-05-04 20:00:17.504 | DEBUG    | napm.config:config_path:28 - /root/.cache/napm/_envs/mmc/config.yaml
2022-05-04 20:00:17.508 | DEBUG    | napm.config:config_path:28 - /root/.cache/napm/_envs/mmc/config.yaml
2022-05-04 20:00:17.513 | DEBUG    | mmc.loaders.fairsliploader:model_factory_from_id:57 - SIMCLR_VITS16
2022

[clip - facebookresearch - clip_small_25ep]
'NoneType' object is not callable


...

[simclr - facebookresearch - simclr_large_25ep]
'NoneType' object is not callable


2022-05-04 20:01:19.681 | DEBUG    | mmc.loaders.fairsliploader:load:172 - {'ssl_emb_dim': 256, 'ssl_mlp_dim': 4096}
2022-05-04 20:01:19.684 | DEBUG    | mmc.loaders.fairsliploader:_napm_install:146 - using napm to "install" facebookresearch/SLIP
2022-05-04 20:01:19.689 | WARNING  | napm.pseudo_install:make_install_dir:21 - /root/.cache/napm/_envs/mmc/SLIP already exists
2022-05-04 20:01:19.814 | DEBUG    | napm.utils:gitclone:22 - 
2022-05-04 20:01:19.819 | DEBUG    | napm.pseudo_install:pseudoinstall_git_repo:62 - Added /root/.cache/napm/_envs/mmc/SLIP to sys.path
2022-05-04 20:01:19.822 | DEBUG    | napm.config:config_path:28 - /root/.cache/napm/_envs/mmc/config.yaml
2022-05-04 20:01:19.827 | DEBUG    | napm.config:config_path:28 - /root/.cache/napm/_envs/mmc/config.yaml
2022-05-04 20:01:19.834 | DEBUG    | napm.config:config_path:28 - /root/.cache/napm/_envs/mmc/config.yaml
2022-05-04 20:01:19.839 | DEBUG    | mmc.loaders.fairsliploader:model_factory_from_id:57 - CLIP_VITB16
2022-0

[clip - facebookresearch - clip_base_cc3m_40ep]
'NoneType' object is not callable


2022-05-04 20:01:27.992 | DEBUG    | mmc.loaders.fairsliploader:load:172 - {'ssl_emb_dim': 256, 'ssl_mlp_dim': 4096}


[clip - facebookresearch - clip_base_cc12m_35ep]
'NoneType' object is not callable
